In [1]:
import torch as torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import ssl
import certifi
ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(), # Convertit image → tenseur
    transforms.Normalize((0.1307,), (0.3081,)) # Normalise les pixels
])

train_data = datasets.MNIST(root="data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="data", train=False, download=True, transform=transform)

train_loader = DataLoader(
  train_data, 
  batch_size=64,  # 64 images à la fois
  shuffle=True,   # Mélange les données pour pas que le modèle
  num_workers=4,  # 4 processus pour charger
  persistent_workers=True, # pas redémarrés
)

test_loader = DataLoader(
  test_data, 
  batch_size=64, 
  shuffle=False, 
  num_workers=4, 
  persistent_workers=True
)

In [3]:
class CNN(nn.Module):
    def __init__(self):
        # Couches Convolutives
        super().__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), # 1, 28 28 -> 32, 28, 28
            nn.ReLU(), # remplace les valeurs négatives par 0
            nn.MaxPool2d(2, 2), #32, 28, 28 -> 32, 14, 14
            nn.Conv2d(32, 64, kernel_size=3, padding=1), # 32, 14, 14 -> 64, 14, 14
            nn.ReLU(),
            nn.MaxPool2d(2, 2), #64, 14, 14 -> 64, 7, 7
        )
        self.fc_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_stack(x)
        x = self.fc_stack(x)
        return x

In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

model = CNN().to(device)

Using mps device


In [5]:
# Configuration de l'entraînement
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
# Entraînement 

epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)

Epoch 1
loss: 2.312315  [   64/60000]
loss: 0.228819  [ 6464/60000]
loss: 0.279069  [12864/60000]
loss: 0.038652  [19264/60000]
loss: 0.084732  [25664/60000]
loss: 0.053000  [32064/60000]
loss: 0.042731  [38464/60000]
loss: 0.049807  [44864/60000]
loss: 0.045292  [51264/60000]
loss: 0.013926  [57664/60000]
Test Error: 
 Accuracy: 98.5%, Avg loss: 0.044703 

Epoch 2
loss: 0.023414  [   64/60000]
loss: 0.012811  [ 6464/60000]
loss: 0.022245  [12864/60000]
loss: 0.026791  [19264/60000]
loss: 0.055098  [25664/60000]
loss: 0.016164  [32064/60000]
loss: 0.069015  [38464/60000]
loss: 0.027698  [44864/60000]
loss: 0.036347  [51264/60000]
loss: 0.005238  [57664/60000]
Test Error: 
 Accuracy: 99.0%, Avg loss: 0.032148 

Epoch 3
loss: 0.035291  [   64/60000]
loss: 0.008824  [ 6464/60000]
loss: 0.127034  [12864/60000]
loss: 0.006535  [19264/60000]
loss: 0.015148  [25664/60000]
loss: 0.000957  [32064/60000]
loss: 0.005969  [38464/60000]
loss: 0.014824  [44864/60000]
loss: 0.029610  [51264/60000]
lo

In [9]:
# Sauvegarde

torch.save(model.state_dict(), "model.pth")

In [10]:
model = CNN().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [11]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.unsqueeze(0).to(device)
    pred = model(x)
    predicted, actual = pred.argmax(1).item(), y

In [12]:
# Recrée le modèle et charge les poids
model = CNN().to('cpu') # export plus sûr avec CPU
model.load_state_dict(torch.load("model.pth", map_location="cpu"))
model.eval()

# Exemple d'entrée (batch de 1 image, 1 canal, 28x28)
dummy_input = torch.randn(1, 1, 28, 28)

# Export en ONNX
torch.onnx.export(
    model,
    dummy_input,
    "model.onnx",
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=17
)